In [47]:
import pandas as pd
import constantes
from tratamento_de_arquivos_e_dados.tratador_de_arquivos import tratador_de_arquivos

In [48]:
df_rat_23 = pd.read_sql('select * from tbl_rat', 'sqlite:///gdo.db')

In [64]:
df_rat_23_pog = df_rat_23[
    ( df_rat_23['NAT.CODIGO'].isin(constantes.NATUREZAS_POG) ) &    
    ( ~ df_rat_23['NOM_UNID_RESPONSAVEL'].str.contains('IND PE', na=False) ) &
    ( ~ df_rat_23['NOM_UNID_RESPONSAVEL'].str.contains('CPE', na=False) ) &
    ( ~ df_rat_23['NOM_UNID_RESPONSAVEL'].str.contains('PRV', na=False) ) &
    ( ~ df_rat_23['NOM_UNID_RESPONSAVEL'].str.contains('MAMB', na=False) ) &
    ( df_rat_23['ANO'] == 2021 )
]

In [65]:
def classifica_cia_por_unid_area(row):
    if '51 CIA' in row['NOM_UNIDADE_AREA']:
        return '51 CIA'
    elif '53 CIA' in row['NOM_UNIDADE_AREA']:
        return '53 CIA'
    elif '139 CIA' in row['NOM_UNIDADE_AREA']:
        return '139 CIA'
    elif '142 CIA' in row['NOM_UNIDADE_AREA']:
        return '142 CIA'
    elif row['MUNICIPIO'] in ['CARMO DO CAJURU', 'SAO GONCALO DO PARA']:
        return '142 CIA'
    elif row['MUNICIPIO'] in ['ITAUNA', 'ITATIAIUCU']:
        return '51 CIA'
    elif row['MUNICIPIO'] == 'CLAUDIO':
        return '139 CIA'
    elif '51 CIA' in row['NOM_UNID_RESPONSAVEL']:
        return '51 CIA'
    elif '53 CIA' in row['NOM_UNID_RESPONSAVEL']:
        return '53 CIA'
    elif '139 CIA' in row['NOM_UNID_RESPONSAVEL']:
        return '139 CIA'
    elif '142 CIA' in row['NOM_UNID_RESPONSAVEL']:
        return '142 CIA'
    else:
        return 'INDEFINIDA'
df_rat_23_pog['CIA_UNIDADE_DE_AREA'] = df_rat_23_pog.apply(lambda row: classifica_cia_por_unid_area(row), axis=1)

<ipython-input-65-92fb24192551>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rat_23_pog['CIA_UNIDADE_DE_AREA'] = df_rat_23_pog.apply(lambda row: classifica_cia_por_unid_area(row), axis=1)


In [66]:
df_rat_23_pog[
    df_rat_23_pog['CIA_UNIDADE_DE_AREA'] == 'INDEFINIDA'
][
    [
        'RAT.NUM_ATIVIDADE', 'TIPO_LOGRADOURO',
        'LOGRADOURO', 'DES_ENDERECO', 'NUM_ENDERECO', 'COMPLEMENTO_ALFA',
        'COMPLEMENTO_ENDERECO', 'NUM_COMPLEMENTAR', 'COD_BAIRRO', 'NOME_BAIRRO',
        'TIPO_LOGRADOURO2', 'LOGRADOURO2', 'DES_ENDERECO2',
        'MUNICIPIO', 'NOM_UNID_RESPONSAVEL']
]

,RAT.NUM_ATIVIDADE,TIPO_LOGRADOURO,LOGRADOURO,DES_ENDERECO,NUM_ENDERECO,COMPLEMENTO_ALFA,COMPLEMENTO_ENDERECO,NUM_COMPLEMENTAR,COD_BAIRRO,NOME_BAIRRO,TIPO_LOGRADOURO2,LOGRADOURO2,DES_ENDERECO2,MUNICIPIO,NOM_UNID_RESPONSAVEL
1947,2021-002644793-001,AVENIDA,PARANA,PARANA,1797,,,,,,,,,DIVINOPOLIS,240 CIA TM/23 BPM/7 RPM
1948,2021-002645612-001,RUA,CRISTO REDENTOR,CRISTO REDENTOR,1681,,,,,,,,,DIVINOPOLIS,240 CIA TM/23 BPM/7 RPM
1950,2021-002646933-001,RUA,SANTA CLARA,SANTA CLARA,660,,,,,,,,,DIVINOPOLIS,240 CIA TM/23 BPM/7 RPM
4610,2021-005901823-001,ACESSO RURAL,,CHACARAS BELO HORIZONTE,,,,,,,,,,DIVINOPOLIS,240 CIA TM/23 BPM/7 RPM
6688,2021-009110130-001,ACESSO RURAL,,CHACARAS BH,0,,,,,,,,,DIVINOPOLIS,23 BPM/7 RPM


In [67]:
classificador_ultimos = {
    '2021-002644793-001': '139 CIA',
    '2021-002645612-001': '139 CIA',
    '2021-002646933-001': '142 CIA',
    '2021-005901823-001': '142 CIA',
    '2021-009110130-001': '142 CIA'
}

In [68]:
def classifica_ultimos(row):
    if row['CIA_UNIDADE_DE_AREA'] == 'INDEFINIDA':
        return classificador_ultimos[row['RAT.NUM_ATIVIDADE']]
    else:
        return row['CIA_UNIDADE_DE_AREA']

df_rat_23_pog['CIA_UNIDADE_DE_AREA'] = df_rat_23_pog.apply(lambda row: classifica_ultimos(row), axis=1)

<ipython-input-68-3ea280e0285d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rat_23_pog['CIA_UNIDADE_DE_AREA'] = df_rat_23_pog.apply(lambda row: classifica_ultimos(row), axis=1)


In [69]:
pd.pivot_table(df_rat_23_pog,index='CIA_UNIDADE_DE_AREA', columns=['ANO', 'MES'], values='RAT.NUM_ATIVIDADE', aggfunc='count')

ANO                 2021     
MES                    1    2
CIA_UNIDADE_DE_AREA          
139 CIA              482  257
142 CIA              588  326
51 CIA               325  267
53 CIA               331  175